In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import sys

In [2]:
with open("anna.txt", 'r') as fin:
    f = fin.read()
f_set = set(f)
print (len(f_set))
vocab2id, id2vocab = {}, {}
vocab2id = {w : i for i, w in enumerate(f_set)}
id2vocab = dict(enumerate(f_set))

83


In [3]:
print (f[0:10])

Chapter 1



In [4]:
encoded = np.array([vocab2id[w] for w in f], dtype = np.int32)

In [5]:
print (encoded[:10])

[46 72 69 22 42 77  4 29 56 28]


In [ ]:
def get_batch(input_data, batch_size, seq_step, num_classes):
    totalc = batch_size * seq_step
    all_batch = len(input_data) // totalc
    indata_len = all_batch * totalc
    indata = input_data[0:indata_len]
    for i in range(all_batch):
        batch_data = indata[i:i+totalc]
        target_data = indata[i+1:i+1+totalc]
        nbatch_data = np.reshape(batch_data, [batch_size, seq_step])
        ntarget_data = np.reshape(batch_data, [batch_size, seq_step])
        
        y = np.zeros(nbatch_data.shape, dtype = nbatch_data.dtype)
        y[:, :ntarget_data.shape[1]] = ntarget_data
        
        nbatch = tf.one_hot(nbatch_data, num_classes).eval()
        y = tf.one_hot(y, num_classes).eval()
        #print ("sss")
        #print (nbatch)
        #print (y)
        yield nbatch, y

num_classes = len(f_set)
batch_size = 10
seq_step = 50
a = get_batch(encoded, batch_size, seq_step, num_classes)

In [ ]:
def build_input(batch_size, seq_step, ndim=0):
    if ndim==0:
        input_data = tf.placeholder(tf.float32, [batch_size, seq_step], name="input_data")
        target_data = tf.placeholder(tf.float32, [batch_size, seq_step], name="output_data")
    else:
        input_data = tf.placeholder(tf.float32, [batch_size, seq_step, ndim], name="input_data")
        target_data = tf.placeholder(tf.float32, [batch_size, seq_step, ndim], name="output_data")
    keep_prob = tf.placeholder(tf.float32, name= "keep_prob")
    
    return input_data, target_data, keep_prob

build_input(10, 50, 83)

In [ ]:
def build_lstm(num_layer, lstm_size, batch_size, keep_prob):
    def build_cell(lstm_size, keep_prob):
        cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop_cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob = keep_prob)
        return drop_cell
    lstm_cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layer)])
    initial_state = lstm_cell.zero_state(batch_size, dtype= tf.float32) ##?????
    return lstm_cell, initial_state
build_lstm(3, 100, 10, 0.8)

In [ ]:
def build_final_output(outputs, lstm_size, outsize):
    x = tf.reshape(outputs, [-1, lstm_size])
    with tf.variable_scope("softmax"):
        w = tf.Variable(tf.truncated_normal([lstm_size, outsize], stddev = 0.1))
        b = tf.Variable(tf.zeros(outsize))
    logits  = tf.matmul(x, w) + b
    out = tf.nn.softmax(logits, name= "pred")
    
    return logits, out

In [ ]:
def build_optimizer(loss, learning_rate, grad_clip):##??
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()  ##????
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [ ]:
def build_loss(logits, target):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = target)
    loss = tf.reduce_mean(loss)  ##?why mean, not sum
    return loss

In [ ]:
##build struc
class charRNN():
    ##?grad_clip?????
    def __init__(self, batch_size, seq_step, num_classes, num_layer=3, lstm_size=100, keep_prob=0.9, learning_rate=0.1, grad_clip=5):
        tf.reset_default_graph()
        lstm_cell, initial_state = build_lstm(num_layer, lstm_size, batch_size, keep_prob)
        self.input_data, self.output_data, self.keep_prob = build_input(batch_size, seq_step, 0)
        self.input_data2, self.output_data2, self.keep_prob2 = build_input(batch_size, seq_step, num_classes)
        #print (self.input_data)
        #print (self.input_data2)
        #self.input_data1 = tf.one_hot(self.input_data, dim, dtype=tf.float32)
        #print (self.input_data1)
        #print (self.input_data2)
        outputs, states = tf.nn.dynamic_rnn(lstm_cell, self.input_data2, initial_state=initial_state, scope="rnntest")
        logits, out = build_final_output(outputs, lstm_size, num_classes)
        self.loss = build_loss(logits, self.output_data2)
        self.opt = build_optimizer(self.loss, learning_rate, grad_clip)
#charRNN(10, 50, 3, 100, 0.9, 83)
        

In [ ]:
epchos = 10
batch_size = 10
seq_step = 50
num_classes = len(f_set)
keep_prob = 0.8

model = charRNN(batch_size, seq_step, num_classes)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(epchos):
        for x, y in get_batch(encoded, batch_size, seq_step, num_classes):
            print (x)
            print (y)
            feed = {model.input_data2:x, model.output_data2:y, model.keep_prob2:keep_prob}
            loss, opt = sess.run([model.loss, model.opt], feed_dict=feed)
            sys.exit()
        